### 아래에서 필수로 필요로 하는 값(없을 경우, 더미데이터로 들어감)

| 컬럼명               | 의미                | 예시 값    |
| ----------------- | ----------------- | ------- |
| `industry_code`   | 업종 코드 (DART KSIC) | `"C26"` |
| `debt_ratio`      | 부채비율(%)           | `120`   |
| `equity_ratio`    | 자기자본비율(%)         | `45`    |
| `current_ratio`   | 유동비율(%)           | `150`   |
| `op_margin`       | 영업이익률(%)          | `12`    |
| `roe`             | 자기자본이익률(ROE, %)   | `10`    |
| `revenue_growth`  | 매출성장률(%)          | `8`     |
| `op_income_prev`  | 전년도 영업이익 (억 원)    | `100`   |
| `op_income_cur`   | 당기 영업이익 (억 원)     | `120`   |
| `net_income_t3`   | 3년 전 순이익 (억 원)    | `70`    |
| `net_income_cur`  | 당기 순이익 (억 원)      | `100`   |
| `avg_salary_prev` | 전년도 평균 인건비 (백만 원) | `50`    |
| `avg_salary_cur`  | 당기 평균 인건비 (백만 원)  | `52`    |


In [20]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from pathlib import Path
from __future__ import annotations
from typing import Dict, Iterable, Optional

# DART / KSIC 업종 코드 → 업종명 매핑 (대표적인 예시)
INDUSTRY_CODE_MAP = {
    "A01": "농업", "A02": "임업", "A03": "어업",
    "B05": "석탄 광업", "B06": "원유 및 천연가스 채굴업", "B07": "금속 광업", "B08": "비금속 광물 광업",
    "C10": "식료품 제조업", "C11": "음료 제조업", "C13": "섬유제품 제조업",
    "C14": "의복, 의복액세서리 및 모피제품 제조업", "C17": "종이 및 종이제품 제조업",
    "C20": "화학물질 및 화학제품 제조업", "C21": "의약품 제조업",
    "C22": "고무제품 및 플라스틱제품 제조업", "C23": "비금속광물제품 제조업",
    "C24": "1차 금속 제조업", "C25": "금속가공제품 제조업",
    "C26": "전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",
    "C27": "의료, 정밀, 광학기기 및 시계 제조업", "C28": "전기장비 제조업",
    "C29": "자동차 및 트레일러 제조업", "C30": "기타 운송장비 제조업", "C31": "가구 제조업",
    "G45": "자동차 및 부품 판매업", "G46": "도매업", "G47": "소매업",
    "J58": "출판업", "J59": "영상, 오디오 기록물 제작 및 배급업", "J60": "방송업",
    "J61": "통신업", "J62": "컴퓨터 프로그래밍, 시스템 통합 및 관리업", "J63": "정보서비스업",
    "K64": "금융업", "K65": "보험 및 연금업", "K66": "금융 및 보험 관련 서비스업",
    "L68": "부동산업", "M70": "전문, 과학 및 기술 서비스업", "M72": "연구개발업",
    "Q86": "보건업", "R90": "예술, 스포츠 및 여가관련 서비스업", "S94": "협회 및 단체",
}

# -*- coding: utf-8 -*-
"""
Industry Average Composite Scores
- 특정 업종코드에 대해 안정성/유동성/수익성/성장성/인적효율성 평균값을 계산
- 데이터가 없으면 모든 업종코드를 커버하는 샘플 DF를 자동 생성하여 즉시 테스트 가능
- 백엔드 연동을 위해 함수 단위로 모듈화

[입력 데이터 컬럼(필수)]
industry_code (업종코드, 문자열)
debt_ratio (%), equity_ratio (%), current_ratio (%), op_margin (%), roe (%), revenue_growth (%)
op_income_prev, op_income_cur, net_income_t3, net_income_cur (양수 실수: 같은 화폐단위/스케일)
avg_salary_prev, avg_salary_cur (양수 실수: 상대 단위 허용)

[수식]
- Stability       = 100 - (부채비율/3) + (자기자본비율/2)         → 0~100 클리핑
- Liquidity       = min(유동비율/2, 100)
- Profitability   = (영업이익률×0.6) + (ROE×0.4)                   → 0~100 클리핑
- Growth          = 0.3×매출성장률 + 0.4×영업이익성장률 + 0.3×순이익CAGR(3y) → [0,100]
- HR_Productivity = 50 + (매출성장률 - 평균인건비성장률)            → 0~100 클리핑
"""



'\nIndustry Average Composite Scores\n- 특정 업종코드에 대해 안정성/유동성/수익성/성장성/인적효율성 평균값을 계산\n- 데이터가 없으면 모든 업종코드를 커버하는 샘플 DF를 자동 생성하여 즉시 테스트 가능\n- 백엔드 연동을 위해 함수 단위로 모듈화\n\n[입력 데이터 컬럼(필수)]\nindustry_code (업종코드, 문자열)\ndebt_ratio (%), equity_ratio (%), current_ratio (%), op_margin (%), roe (%), revenue_growth (%)\nop_income_prev, op_income_cur, net_income_t3, net_income_cur (양수 실수: 같은 화폐단위/스케일)\navg_salary_prev, avg_salary_cur (양수 실수: 상대 단위 허용)\n\n[수식]\n- Stability       = 100 - (부채비율/3) + (자기자본비율/2)         → 0~100 클리핑\n- Liquidity       = min(유동비율/2, 100)\n- Profitability   = (영업이익률×0.6) + (ROE×0.4)                   → 0~100 클리핑\n- Growth          = 0.3×매출성장률 + 0.4×영업이익성장률 + 0.3×순이익CAGR(3y) → [0,100]\n- HR_Productivity = 50 + (매출성장률 - 평균인건비성장률)            → 0~100 클리핑\n'

In [21]:

# =========================
# 설정값(필요 시 한곳에서만 수정)
# =========================
CSV_PATH: Optional[str] = None       # 예: "dart_extracted.csv" (없으면 샘플 자동 생성)
INDUSTRY_COL = "industry_code"       # 업종 식별 컬럼명
N_ROWS_PER_INDUSTRY = 6              # 샘플 생성 시 업종당 레코드 수
RANDOM_SEED = 42                     # 샘플 생성 재현성

# 필수 컬럼
REQUIRED_COLS = [
    INDUSTRY_COL,
    "debt_ratio", "equity_ratio",
    "current_ratio",
    "op_margin", "roe",
    "revenue_growth",
    "op_income_prev", "op_income_cur",
    "net_income_t3", "net_income_cur",
    "avg_salary_prev", "avg_salary_cur",
]


In [24]:


# =========================
# 유틸 & 수식 모듈
# =========================
def _clip_range(x: float, low: float = 0.0, high: float = 100.0) -> float:
    return float(np.clip(x, low, high))

def _nan_to_num(x: Optional[float], fill: float = 0.0) -> float:
    return fill if x is None or (isinstance(x, float) and pd.isna(x)) else float(x)

def _pct_change(prev: Optional[float], cur: Optional[float]) -> float:
    """(cur - prev) / |prev| × 100  (prev==0이면 증가 시 100, 동일/감소 시 0으로 처리)"""
    if prev is None or cur is None or pd.isna(prev) or pd.isna(cur):
        return np.nan
    if prev == 0:
        return 100.0 if cur > 0 else 0.0
    return float((cur - prev) / abs(prev) * 100.0)

def _cagr(start: Optional[float], end: Optional[float], years: int) -> float:
    """CAGR = (end/start)^(1/years) - 1 → %"""
    if start is None or end is None or pd.isna(start) or pd.isna(end) or start <= 0 or years <= 0:
        return np.nan
    try:
        return float(((end / start) ** (1.0 / years) - 1.0) * 100.0)
    except Exception:
        return np.nan

# ---- Score functions (요청한 수식 1:1 반영) ----

#안정성
def score_stability(debt_ratio: Optional[float], equity_ratio: Optional[float]) -> float:
    raw = 100.0 - _nan_to_num(debt_ratio)/3.0 + _nan_to_num(equity_ratio)/2.0
    return _clip_range(raw)

#유동성
def score_liquidity(current_ratio: Optional[float]) -> float:
    if current_ratio is None or pd.isna(current_ratio):
        return 0.0
    return _clip_range(current_ratio/2.0)

#수익성
def score_profitability(op_margin: Optional[float], roe: Optional[float]) -> float:
    raw = _nan_to_num(op_margin)*0.6 + _nan_to_num(roe)*0.4
    return _clip_range(raw)

#성장성
def score_growth(revenue_growth: Optional[float],
                 op_income_prev: Optional[float], op_income_cur: Optional[float],
                 net_income_t3: Optional[float], net_income_cur: Optional[float]) -> float:
    op_g = _pct_change(op_income_prev, op_income_cur)
    ni_cagr = _cagr(net_income_t3, net_income_cur, 3)
    raw = 0.3*_nan_to_num(revenue_growth) + 0.4*_nan_to_num(op_g) + 0.3*_nan_to_num(ni_cagr)
    return _clip_range(raw, 0.0, 100.0)

#인적투입 효율성(인적효율성)
def score_hr_productivity(avg_salary_prev: Optional[float], avg_salary_cur: Optional[float],
                          revenue_growth: Optional[float]) -> float:
    sal_g = _pct_change(avg_salary_prev, avg_salary_cur)
    raw = 50.0 + _nan_to_num(revenue_growth) - _nan_to_num(sal_g)
    return _clip_range(raw)

# =========================
# 퍼블릭 API
# =========================
def validate_columns(df: pd.DataFrame, required: Iterable[str] = REQUIRED_COLS) -> None:
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"필수 컬럼 누락: {missing}")

def compute_company_scores(df: pd.DataFrame) -> pd.DataFrame:
    """
    회사별 점수 컬럼 추가:
    - Stability, Liquidity, Profitability, Growth, HR_Productivity
    """
    validate_columns(df)
    out = df.copy()
    out["Stability"]       = out.apply(lambda r: score_stability(r["debt_ratio"], r["equity_ratio"]), axis=1)
    out["Liquidity"]       = out.apply(lambda r: score_liquidity(r["current_ratio"]), axis=1)
    out["Profitability"]   = out.apply(lambda r: score_profitability(r["op_margin"], r["roe"]), axis=1)
    out["Growth"]          = out.apply(lambda r: score_growth(r["revenue_growth"], r["op_income_prev"], r["op_income_cur"],
                                                              r["net_income_t3"], r["net_income_cur"]), axis=1)
    out["HR_Productivity"] = out.apply(lambda r: score_hr_productivity(r["avg_salary_prev"], r["avg_salary_cur"],
                                                                       r["revenue_growth"]), axis=1)
    return out

def get_industry_average(df: pd.DataFrame, industry_code: str) -> Dict[str, float]:
    """
    특정 업종코드 평균값 반환(소수점 1자리 반올림)
    반환 예: {'Stability': 83.3, 'Liquidity': 78.3, 'Profitability': 11.5, 'Growth': 7.9, 'HR_Productivity': 49.9}
    """
    validate_columns(df)
    df_f = df[df[INDUSTRY_COL] == industry_code]
    if df_f.empty:
        raise ValueError(f"❌ 업종코드 {industry_code} 데이터 없음")
    scored = compute_company_scores(df_f)
    return scored[["Stability","Liquidity","Profitability","Growth","HR_Productivity"]].mean().round(1).to_dict()

def get_all_industry_averages(df: pd.DataFrame) -> pd.DataFrame:
    """
    DF 내 존재하는 모든 업종코드에 대해 평균표 반환
    컬럼: industry_code, Stability, Liquidity, Profitability, Growth, HR_Productivity
    """
    validate_columns(df)
    scored = compute_company_scores(df)
    grouped = (
        scored.groupby(INDUSTRY_COL)[["Stability","Liquidity","Profitability","Growth","HR_Productivity"]]
              .mean()
              .reset_index()
              .round(1)
    )
    return grouped

# =========================
# 데이터 로딩 & 샘플 생성
# =========================
def synthesize_sample_df(codes: Optional[Iterable[str]] = None,
                         n_per_industry: int = N_ROWS_PER_INDUSTRY,
                         seed: int = RANDOM_SEED) -> pd.DataFrame:
    """
    실데이터가 없을 때 업종코드 전부 커버하는 샘플 DF 생성(재현성 보장)
    업종 접두(제조/ICT/금융/유통 등)에 따라 대략적인 분포 차이를 둠
    """
    np.random.seed(seed)
    rows = []
    _codes = list(codes) if codes is not None else list(INDUSTRY_CODE_MAP.keys())
    for code in _codes:
        prefix = code[0]
        # 업종군별 대략적 분포
        if prefix == "C":     # 제조업
            op_margin_mu, roe_mu, cr_mu = 12, 11, 170
        elif prefix == "J":   # 정보통신/콘텐츠
            op_margin_mu, roe_mu, cr_mu = 10, 9, 160
        elif prefix == "K":   # 금융
            op_margin_mu, roe_mu, cr_mu = 9, 10, 130
        elif prefix == "G":   # 도소매/유통
            op_margin_mu, roe_mu, cr_mu = 8, 8, 150
        elif prefix == "L":   # 부동산
            op_margin_mu, roe_mu, cr_mu = 7, 7, 120
        else:                 # 기타
            op_margin_mu, roe_mu, cr_mu = 8, 7, 140

        for _ in range(n_per_industry):
            debt_ratio   = np.clip(np.random.normal(130, 40), 50, 260)      # %
            equity_ratio = np.clip(np.random.normal(40, 10), 10, 70)        # %
            current_ratio= np.clip(np.random.normal(cr_mu, 40), 70, 350)    # %
            op_margin    = np.clip(np.random.normal(op_margin_mu, 4), 2, 25)# %
            roe          = np.clip(np.random.normal(roe_mu, 4), 2, 22)      # %
            revenue_g    = np.clip(np.random.normal(8, 8), -5, 25)          # %

            base   = np.random.uniform(50, 300)
            op_prev= base
            op_cur = base * (1 + revenue_g/100 * np.random.uniform(0.6, 1.2))
            ni_t3  = base * np.random.uniform(0.6, 1.0)
            ni_cur = op_cur * np.random.uniform(0.6, 1.0)

            sal_prev = np.random.uniform(35, 70)
            sal_cur  = sal_prev * np.random.uniform(0.98, 1.10)

            rows.append({
                INDUSTRY_COL: code,
                "debt_ratio": float(debt_ratio),
                "equity_ratio": float(equity_ratio),
                "current_ratio": float(current_ratio),
                "op_margin": float(op_margin),
                "roe": float(roe),
                "revenue_growth": float(revenue_g),
                "op_income_prev": float(op_prev),
                "op_income_cur":  float(op_cur),
                "net_income_t3":  float(ni_t3),
                "net_income_cur": float(ni_cur),
                "avg_salary_prev": float(sal_prev),
                "avg_salary_cur":  float(sal_cur),
            })
    return pd.DataFrame(rows)

def load_dataframe(csv_path: Optional[str] = CSV_PATH) -> pd.DataFrame:
    """
    CSV 경로가 주어지면 로드, 없으면 전 업종 샘플 DF 자동 생성
    """
    if csv_path and Path(csv_path).exists():
        return pd.read_csv(csv_path)
    return synthesize_sample_df()

# =========================
# CLI (단독 실행 시)
# =========================
def _print_industry_average(df: pd.DataFrame, code: str) -> None:
    avg = get_industry_average(df, code)
    name = INDUSTRY_CODE_MAP.get(code)
    label = f"{code} ({name})" if name else code
    print(f"\n[{label}] 업종 평균 종합지표")
    print(f"- 안정성: {avg['Stability']}")
    print(f"- 유동성: {avg['Liquidity']}")
    print(f"- 수익성: {avg['Profitability']}")
    print(f"- 성장성: {avg['Growth']}")
    print(f"- 인적효율성: {avg['HR_Productivity']}")

if __name__ == "__main__":
    df = load_dataframe(CSV_PATH)
    code = input("🔢 업종코드를 입력하세요 (예: C26): ").strip()
    try:
        _print_industry_average(df, code)
    except Exception as e:
        print("오류:", e)


🔢 업종코드를 입력하세요 (예: C26): K64

[K64 (금융업)] 업종 평균 종합지표
- 안정성: 75.9
- 유동성: 56.6
- 수익성: 11.2
- 성장성: 8.9
- 인적효율성: 60.0
